# Module 05: Hooks and Automation

**Difficulty**: ⭐⭐ Intermediate  
**Estimated Time**: 150 minutes  
**Prerequisites**: [Module 03 - Skills](03_working_with_skills.ipynb), [Module 04 - Commands](04_custom_slash_commands.ipynb)

## Learning Objectives

By the end of this module, you will be able to:

1. Understand hook types and triggering events
2. Create validation hooks to prevent mistakes
3. Implement automated quality gates
4. Debug hook failures effectively
5. Follow hook best practices
6. Build a complete pre-commit workflow

---

## 1. Introduction to Hooks

### What are Hooks in Claude Code?

**Hooks** are automated scripts that run in response to specific events in Claude Code. They allow you to:
- **Validate** operations before they execute
- **Block** dangerous or unwanted actions
- **Automate** repetitive quality checks
- **Enforce** team standards and policies

Think of hooks as **guardrails** and **automation triggers** for your workflow.

### Event-Driven Automation

Hooks work on an **event-driven model**:

```
Event occurs → Hook triggers → Script runs → Allow or Block
```

**Example**: Before Claude writes to a file, a hook can:
- Check if the file is critical (e.g., `production.json`)
- Block the write if it's dangerous
- Show a helpful error message

### Hook Types

Claude Code supports three hook events:

| Hook Event | When It Triggers | Use Cases |
|------------|------------------|----------|
| **tool_call** | Before/after tool execution | Block file writes, validate commands |
| **user_prompt_submit** | When user sends a message | Log conversations, enforce prompts |
| **assistant_message** | When Claude responds | Track responses, save outputs |

### When Hooks Trigger

**tool_call** (most common):
```bash
User: "Delete the config file"
  ↓
Claude tries to use Bash tool
  ↓
Hook runs BEFORE Bash executes
  ↓
Hook can block (exit 1) or allow (exit 0)
```

**user_prompt_submit**:
```bash
User types a message and hits Enter
  ↓
Hook runs BEFORE message sent to Claude
  ↓
Can validate/modify/log the message
```

**assistant_message**:
```bash
Claude generates a response
  ↓
Hook runs AFTER response generated
  ↓
Can log, analyze, or save responses
```

### Real-World Use Cases

**Validation**:
- Prevent writes to production files
- Block deletion of critical directories
- Require tests before commits

**Automation**:
- Auto-format code before saving
- Run linters on file changes
- Generate documentation automatically

**Quality Gates**:
- Enforce test coverage thresholds
- Check code style compliance
- Validate security requirements

---

## 2. Hook Architecture

### Bash Scripts with Exit Codes

Hooks are **bash scripts** that communicate via exit codes:

- **Exit 0**: ✅ Allow the operation to proceed
- **Exit 1**: ❌ Block the operation

**Example Hook Script**:
```bash
#!/bin/bash

# Check if operation is allowed
if [ "$TOOL_NAME" = "Bash" ]; then
    echo "⚠️  Bash commands require approval"
    exit 1  # Block
fi

exit 0  # Allow all other operations
```

### Environment Variables

Hooks receive information through environment variables:

| Variable | Description | Example |
|----------|-------------|--------|
| `$HOOK_EVENT` | Event type | `tool_call`, `user_prompt_submit` |
| `$TOOL_NAME` | Tool being used | `Bash`, `Write`, `Edit` |
| `$TOOL_ARGS` | Tool arguments (JSON) | `{"command": "ls -la"}` |
| `$PROMPT` | User's message | (for user_prompt_submit) |
| `$RESPONSE` | Claude's response | (for assistant_message) |

### Configuration in settings.json

Hooks are configured in `.claude/settings.json`:

```json
{
  "hooks": {
    "tool_call": {
      "command": ".claude/hooks/validate-tool.sh"
    },
    "user_prompt_submit": {
      "command": ".claude/hooks/log-prompts.sh"
    }
  }
}
```

### Hook Execution Flow

```
1. Event occurs (e.g., tool call)
     ↓
2. Claude Code checks settings.json for hook
     ↓
3. If hook exists, run the bash script
     ↓
4. Script receives environment variables
     ↓
5. Script executes and returns exit code
     ↓
6. Exit 0 → Allow | Exit 1 → Block with error message
```

### File Structure

**Recommended organization**:
```
.claude/
├── hooks/
│   ├── validate-tool.sh       # tool_call hook
│   ├── log-prompts.sh         # user_prompt_submit hook
│   ├── protect-files.sh       # File protection
│   └── pre-commit.sh          # Quality gates
└── settings.json              # Hook configuration
```

---

## 3. Hands-On Section 1: Your First Hook ⭐

Let's create a simple logging hook that records all Bash commands.

### Exercise 1: Create a Bash Command Logger

**Goal**: Log all Bash tool usage to a file.

**Step 1**: Create the hook directory

In [ ]:
import os
from pathlib import Path

# Create hooks directory if it doesn't exist
hooks_dir = Path('.claude/hooks')
hooks_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Created hooks directory: {hooks_dir}")
print(f"   Absolute path: {hooks_dir.resolve()}")

**Step 2**: Create the logging hook script

**File**: `.claude/hooks/log-bash.sh`

```bash
#!/bin/bash

# Log all Bash commands to a file

LOG_FILE=".claude/hooks/bash-commands.log"

# Only log Bash tool calls
if [ "$TOOL_NAME" = "Bash" ]; then
    TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')
    
    # Extract command from TOOL_ARGS (JSON)
    COMMAND=$(echo "$TOOL_ARGS" | grep -o '"command":"[^"]*"' | cut -d'"' -f4)
    
    # Log to file
    echo "[$TIMESTAMP] $COMMAND" >> "$LOG_FILE"
    
    echo "📝 Logged command to $LOG_FILE"
fi

# Always allow the operation
exit 0
```

Let's create this script:

In [ ]:
# Create the logging hook script
hook_script = Path('.claude/hooks/log-bash.sh')

hook_content = '''#!/bin/bash

# Log all Bash commands to a file

LOG_FILE=".claude/hooks/bash-commands.log"

# Only log Bash tool calls
if [ "$TOOL_NAME" = "Bash" ]; then
    TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')
    
    # Extract command from TOOL_ARGS (JSON)
    COMMAND=$(echo "$TOOL_ARGS" | grep -o '"command":"[^"]*"' | cut -d'"' -f4)
    
    # Log to file
    echo "[$TIMESTAMP] $COMMAND" >> "$LOG_FILE"
    
    echo "📝 Logged command to $LOG_FILE"
fi

# Always allow the operation
exit 0
'''

hook_script.write_text(hook_content)

# Make it executable
hook_script.chmod(0o755)

print(f"✅ Created hook script: {hook_script}")
print(f"✅ Made executable (chmod 755)")

**Step 3**: Enable the hook in settings.json

**File**: `.claude/settings.json`

```json
{
  "hooks": {
    "tool_call": {
      "command": ".claude/hooks/log-bash.sh"
    }
  }
}
```

In [ ]:
import json

settings_file = Path('.claude/settings.json')

# Read existing settings or create new
if settings_file.exists():
    settings = json.loads(settings_file.read_text())
else:
    settings = {}

# Add hook configuration
settings['hooks'] = {
    'tool_call': {
        'command': '.claude/hooks/log-bash.sh'
    }
}

# Write updated settings
settings_file.write_text(json.dumps(settings, indent=2))

print("✅ Updated .claude/settings.json")
print("\nConfiguration:")
print(json.dumps(settings, indent=2))

**Step 4**: Test the hook

Now when you use Claude Code and it runs Bash commands, they'll be logged!

**To verify**: Check `.claude/hooks/bash-commands.log` after running some Bash commands.

### Exercise 2: Review the Log

After using the hook for a while, review what was logged:

In [ ]:
log_file = Path('.claude/hooks/bash-commands.log')

if log_file.exists():
    print("📋 Bash Command Log:\n")
    print(log_file.read_text())
    
    # Count entries
    entries = log_file.read_text().strip().split('\n')
    print(f"\n✅ Total commands logged: {len(entries)}")
else:
    print("ℹ️  No log file yet. Run some Bash commands first!")

**Key Takeaways**:
- ✅ Hooks run automatically on events
- ✅ Exit 0 allows operations (logging hook)
- ✅ Environment variables provide context
- ✅ Hooks configured in settings.json

---

## 4. Validation Hooks

### Blocking Operations

Unlike logging hooks that always allow operations (exit 0), **validation hooks** can **block** dangerous actions by returning exit code 1.

### Protecting Critical Files

**Use case**: Prevent accidental writes to production configuration files.

**Pattern**:
```bash
if [ dangerous_condition ]; then
    echo "❌ Error: Operation blocked!"
    exit 1  # Block
fi

exit 0  # Allow
```

### Validation Patterns

**1. File Protection**:
```bash
# Block writes to production.json
if [[ "$TOOL_NAME" = "Write" && "$TOOL_ARGS" =~ "production.json" ]]; then
    echo "❌ Cannot modify production.json!"
    exit 1
fi
```

**2. Command Validation**:
```bash
# Block dangerous rm -rf commands
if [[ "$TOOL_ARGS" =~ "rm -rf /" ]]; then
    echo "❌ Blocked: Dangerous deletion command!"
    exit 1
fi
```

**3. Directory Protection**:
```bash
# Protect .git directory
if [[ "$TOOL_ARGS" =~ "\.git" ]]; then
    echo "⚠️  Warning: Operation affects .git directory"
    exit 1
fi
```

### Helpful Error Messages

When blocking, provide **clear, actionable** error messages:

**Bad**:
```bash
echo "Error"
exit 1
```

**Good**:
```bash
echo "❌ Cannot write to production.json"
echo "   Reason: Protected configuration file"
echo "   Solution: Edit staging.json instead, then promote to production"
exit 1
```

---

## 5. Hands-On Section 2: Validation Hook ⭐⭐

### Exercise 3: Protect Production Files

**Goal**: Block writes to critical configuration files.

**File**: `.claude/hooks/protect-files.sh`

```bash
#!/bin/bash

# Protect critical files from modification

# List of protected files (patterns)
PROTECTED_FILES=(
    "production.json"
    "secrets.env"
    ".env.production"
    "database.config"
)

# Only check Write and Edit tools
if [[ "$TOOL_NAME" = "Write" || "$TOOL_NAME" = "Edit" ]]; then
    
    # Check if any protected file is being modified
    for pattern in "${PROTECTED_FILES[@]}"; do
        if [[ "$TOOL_ARGS" =~ $pattern ]]; then
            echo "❌ BLOCKED: Cannot modify protected file: $pattern"
            echo ""
            echo "   Reason: This is a critical production file"
            echo "   Solution: "
            echo "     1. Edit the staging/development version instead"
            echo "     2. Test thoroughly"
            echo "     3. Promote to production through proper channels"
            echo ""
            exit 1
        fi
    done
fi

# Allow operation
exit 0
```

Let's create this hook:

In [ ]:
protect_hook = Path('.claude/hooks/protect-files.sh')

protect_content = '''#!/bin/bash

# Protect critical files from modification

# List of protected files (patterns)
PROTECTED_FILES=(
    "production.json"
    "secrets.env"
    ".env.production"
    "database.config"
)

# Only check Write and Edit tools
if [[ "$TOOL_NAME" = "Write" || "$TOOL_NAME" = "Edit" ]]; then
    
    # Check if any protected file is being modified
    for pattern in "${PROTECTED_FILES[@]}"; do
        if [[ "$TOOL_ARGS" =~ $pattern ]]; then
            echo "❌ BLOCKED: Cannot modify protected file: $pattern"
            echo ""
            echo "   Reason: This is a critical production file"
            echo "   Solution: "
            echo "     1. Edit the staging/development version instead"
            echo "     2. Test thoroughly"
            echo "     3. Promote to production through proper channels"
            echo ""
            exit 1
        fi
    done
fi

# Allow operation
exit 0
'''

protect_hook.write_text(protect_content)
protect_hook.chmod(0o755)

print(f"✅ Created protection hook: {protect_hook}")
print(f"✅ Made executable")
print("\n🛡️  Protected files:")
print("   - production.json")
print("   - secrets.env")
print("   - .env.production")
print("   - database.config")

### Exercise 4: Test the Protection

Let's test that the hook properly blocks protected files:

In [ ]:
import subprocess
import os

def test_hook(tool_name, file_path):
    """Test hook by simulating environment variables."""
    
    env = os.environ.copy()
    env['TOOL_NAME'] = tool_name
    env['TOOL_ARGS'] = f'{{"file_path": "{file_path}"}}'
    
    result = subprocess.run(
        ['.claude/hooks/protect-files.sh'],
        env=env,
        capture_output=True,
        text=True
    )
    
    return result.returncode, result.stdout

# Test cases
test_cases = [
    ('Write', 'production.json', 'SHOULD BLOCK'),
    ('Write', 'development.json', 'SHOULD ALLOW'),
    ('Edit', 'secrets.env', 'SHOULD BLOCK'),
    ('Edit', 'config.json', 'SHOULD ALLOW'),
]

print("🧪 Testing File Protection Hook:\n")
print("=" * 60)

for tool, file, expected in test_cases:
    code, output = test_hook(tool, file)
    status = "❌ BLOCKED" if code == 1 else "✅ ALLOWED"
    match = "✓" if (code == 1 and 'BLOCK' in expected) or (code == 0 and 'ALLOW' in expected) else "✗"
    
    print(f"\n{match} {tool} '{file}': {status} ({expected})")
    if output:
        print(f"   Output: {output.strip()[:100]}...")

print("\n" + "=" * 60)

### Exercise 5: Customize Protected Files

**Task**: Add your own protected files to the list.

Edit `.claude/hooks/protect-files.sh` and add patterns like:
- `*.prod.js` - All production JavaScript files
- `credentials.*` - Any credentials file
- `master.key` - Master encryption key

**Challenge**: Make the hook case-insensitive for better protection.

---

## 6. Pre-Commit Hooks and Quality Gates

### Quality Gates Concept

**Quality gates** are automated checks that ensure code meets standards before being committed.

**Common quality gates**:
- ✅ Code formatting (prettier, black)
- ✅ Linting (eslint, flake8)
- ✅ Tests pass
- ✅ No TODOs or FIXMEs
- ✅ No secrets in code

### Pre-Commit Workflow

```
Developer writes code
    ↓
Requests commit
    ↓
Pre-commit hook runs
    ↓
┌─────────────────────┐
│ Run Linter          │ → Fails? Block commit
│ Run Tests           │ → Fails? Block commit  
│ Check Formatting    │ → Fails? Block commit
│ Scan for Secrets    │ → Found? Block commit
└─────────────────────┘
    ↓
All checks pass?
    ↓
✅ Allow commit
```

### Hook-Based Quality Automation

In Claude Code, we can implement quality gates using the `tool_call` hook:

```bash
# When Bash tool tries to run 'git commit'
if [[ "$TOOL_ARGS" =~ "git commit" ]]; then
    # Run quality checks
    npm run lint || exit 1
    npm test || exit 1
    
    # All passed, allow commit
    exit 0
fi
```

### Auto-Formatting Pattern

Instead of blocking, we can **automatically fix** issues:

```bash
# Auto-format before saving
if [[ "$TOOL_NAME" = "Write" && "$TOOL_ARGS" =~ "\.py$" ]]; then
    # Extract filename
    FILE=$(echo "$TOOL_ARGS" | grep -o '"file_path":"[^"]*"' | cut -d'"' -f4)
    
    # Format with black (after file is written)
    black "$FILE" 2>/dev/null
    
    echo "✨ Auto-formatted $FILE with black"
fi
```

---

## 7. Hands-On Section 3: Quality Gates ⭐⭐⭐

### Exercise 6: Create a Pre-Commit Linting Hook

**Goal**: Run linter before allowing commits.

**File**: `.claude/hooks/pre-commit-lint.sh`

```bash
#!/bin/bash

# Run linting before git commits

# Check if this is a git commit command
if [[ "$TOOL_NAME" = "Bash" && "$TOOL_ARGS" =~ "git commit" ]]; then
    
    echo "🔍 Running pre-commit checks..."
    echo ""
    
    # Check for Python files
    if ls *.py &>/dev/null; then
        echo "  → Linting Python files with flake8..."
        
        if command -v flake8 &>/dev/null; then
            if ! flake8 *.py --max-line-length=100 --extend-ignore=E203; then
                echo ""
                echo "❌ Linting failed! Fix errors before committing."
                exit 1
            fi
            echo "  ✅ Linting passed"
        else
            echo "  ⚠️  flake8 not installed, skipping"
        fi
    fi
    
    # Check for JavaScript files
    if ls *.js &>/dev/null; then
        echo "  → Linting JavaScript files..."
        
        if command -v eslint &>/dev/null; then
            if ! eslint *.js; then
                echo ""
                echo "❌ ESLint failed! Fix errors before committing."
                exit 1
            fi
            echo "  ✅ ESLint passed"
        else
            echo "  ⚠️  eslint not installed, skipping"
        fi
    fi
    
    echo ""
    echo "✅ All pre-commit checks passed!"
fi

exit 0
```

In [ ]:
precommit_hook = Path('.claude/hooks/pre-commit-lint.sh')

precommit_content = '''#!/bin/bash

# Run linting before git commits

# Check if this is a git commit command
if [[ "$TOOL_NAME" = "Bash" && "$TOOL_ARGS" =~ "git commit" ]]; then
    
    echo "🔍 Running pre-commit checks..."
    echo ""
    
    # Check for Python files
    if ls *.py &>/dev/null; then
        echo "  → Linting Python files with flake8..."
        
        if command -v flake8 &>/dev/null; then
            if ! flake8 *.py --max-line-length=100 --extend-ignore=E203; then
                echo ""
                echo "❌ Linting failed! Fix errors before committing."
                exit 1
            fi
            echo "  ✅ Linting passed"
        else
            echo "  ⚠️  flake8 not installed, skipping"
        fi
    fi
    
    echo ""
    echo "✅ All pre-commit checks passed!"
fi

exit 0
'''

precommit_hook.write_text(precommit_content)
precommit_hook.chmod(0o755)

print(f"✅ Created pre-commit linting hook: {precommit_hook}")
print(f"✅ Made executable")
print("\n🔍 This hook will:")
print("   - Run automatically before git commits")
print("   - Lint Python files with flake8")
print("   - Block commits if linting fails")

### Exercise 7: Create Auto-Formatting Hook

**Goal**: Automatically format Python code on save.

**File**: `.claude/hooks/auto-format.sh`

```bash
#!/bin/bash

# Auto-format Python files when saved

if [[ "$TOOL_NAME" = "Write" || "$TOOL_NAME" = "Edit" ]]; then
    # Extract filename from TOOL_ARGS
    FILE=$(echo "$TOOL_ARGS" | grep -o '"file_path":"[^"]*"' | cut -d'"' -f4)
    
    # Check if it's a Python file
    if [[ "$FILE" =~ \.py$ ]]; then
        if command -v black &>/dev/null; then
            # Format after Claude writes the file
            # Note: This runs AFTER the write, so we need to allow it first
            (
                sleep 0.1  # Let write complete
                black "$FILE" --line-length 100 &>/dev/null
                echo "✨ Auto-formatted $FILE with black"
            ) &
        fi
    fi
fi

exit 0
```

In [ ]:
format_hook = Path('.claude/hooks/auto-format.sh')

format_content = '''#!/bin/bash

# Auto-format Python files when saved

if [[ "$TOOL_NAME" = "Write" || "$TOOL_NAME" = "Edit" ]]; then
    # Extract filename from TOOL_ARGS
    FILE=$(echo "$TOOL_ARGS" | grep -o '"file_path":"[^"]*"' | cut -d'"' -f4)
    
    # Check if it's a Python file
    if [[ "$FILE" =~ \.py$ ]]; then
        if command -v black &>/dev/null; then
            # Format after Claude writes the file
            (
                sleep 0.1  # Let write complete
                black "$FILE" --line-length 100 &>/dev/null
                echo "✨ Auto-formatted $FILE with black"
            ) &
        fi
    fi
fi

exit 0
'''

format_hook.write_text(format_content)
format_hook.chmod(0o755)

print(f"✅ Created auto-format hook: {format_hook}")
print(f"✅ Made executable")
print("\n✨ This hook will:")
print("   - Auto-format Python files with black")
print("   - Run after Write or Edit operations")
print("   - Use max line length of 100")

### Exercise 8: Chain Multiple Checks

**Goal**: Combine multiple quality checks in one hook.

**Task**: Create a comprehensive quality gate hook that:
1. Checks code formatting
2. Runs linter
3. Runs tests
4. Scans for TODO/FIXME comments
5. Checks for secrets (API keys, passwords)

If any check fails, block the commit with a clear error message.

---

## 8. Hook Events Deep Dive

### tool_call Event

**When**: Before or after a tool is executed

**Environment variables**:
- `$HOOK_EVENT` = `"tool_call"`
- `$TOOL_NAME` = Name of tool (Bash, Write, Edit, Read, etc.)
- `$TOOL_ARGS` = JSON string of tool arguments

**Use cases**:
- Validate file operations
- Log tool usage
- Block dangerous commands
- Auto-format code

**Example**:
```bash
# $TOOL_NAME = "Write"
# $TOOL_ARGS = '{"file_path": "config.json", "content": "..."}'
```

### user_prompt_submit Event

**When**: User submits a message to Claude

**Environment variables**:
- `$HOOK_EVENT` = `"user_prompt_submit"`
- `$PROMPT` = The user's message

**Use cases**:
- Log all user prompts
- Analyze prompt patterns
- Enforce prompt templates
- Track feature usage

**Example**:
```bash
# Log all prompts
echo "[$(date)] $PROMPT" >> prompts.log
```

### assistant_message Event

**When**: Claude generates a response

**Environment variables**:
- `$HOOK_EVENT` = `"assistant_message"`
- `$RESPONSE` = Claude's response text

**Use cases**:
- Save responses for review
- Analyze response quality
- Extract code snippets
- Build knowledge base

### Choosing the Right Event

| Goal | Event | Why |
|------|-------|-----|
| Block file writes | `tool_call` | Need to intercept Write tool |
| Log all interactions | `user_prompt_submit` + `assistant_message` | Capture both sides |
| Validate commands | `tool_call` | Check before Bash executes |
| Track usage patterns | `user_prompt_submit` | Analyze what users ask |
| Auto-format code | `tool_call` | React to Write/Edit |

---

## 9. Hook Environment Variables

### Parsing TOOL_ARGS (JSON)

`$TOOL_ARGS` is a JSON string. Extract values using tools like `jq` or regex:

**Using grep (simple)**:
```bash
# Extract file_path
FILE=$(echo "$TOOL_ARGS" | grep -o '"file_path":"[^"]*"' | cut -d'"' -f4)

# Extract command
CMD=$(echo "$TOOL_ARGS" | grep -o '"command":"[^"]*"' | cut -d'"' -f4)
```

**Using jq (robust)**:
```bash
# Extract file_path
FILE=$(echo "$TOOL_ARGS" | jq -r '.file_path')

# Extract command
CMD=$(echo "$TOOL_ARGS" | jq -r '.command')
```

### Available Variables by Event

**tool_call**:
```bash
echo "Hook event: $HOOK_EVENT"
echo "Tool name: $TOOL_NAME"
echo "Tool args: $TOOL_ARGS"
```

**user_prompt_submit**:
```bash
echo "Hook event: $HOOK_EVENT"
echo "User prompt: $PROMPT"
```

**assistant_message**:
```bash
echo "Hook event: $HOOK_EVENT"
echo "Assistant response: $RESPONSE"
```

### Debugging: Print All Variables

```bash
#!/bin/bash

echo "=== Hook Debug ==="
echo "Event: $HOOK_EVENT"
echo "Tool: $TOOL_NAME"
echo "Args: $TOOL_ARGS"
echo "Prompt: $PROMPT"
echo "Response: ${RESPONSE:0:100}..."  # First 100 chars
echo "=================="

exit 0
```

---

## 10. Debugging Hooks

### Common Hook Failures

**1. Permission Denied**
```bash
# Error: Permission denied
# Fix: Make script executable
chmod +x .claude/hooks/my-hook.sh
```

**2. Script Not Found**
```bash
# Error: No such file or directory
# Fix: Check path in settings.json
# Use relative path from project root
".claude/hooks/my-hook.sh"  # ✓ Correct
"hooks/my-hook.sh"          # ✗ Wrong
```

**3. Unexpected Exit Code**
```bash
# Always explicitly exit
exit 0  # Allow
exit 1  # Block

# Don't rely on implicit exit
```

**4. Variable Not Set**
```bash
# Check if variable exists
if [ -z "$TOOL_NAME" ]; then
    echo "Error: TOOL_NAME not set"
    exit 0  # Allow if variable missing
fi
```

### Debugging Techniques

**1. Add Logging**
```bash
#!/bin/bash

LOG=".claude/hooks/debug.log"

# Log everything
{
    echo "=== $(date) ==="
    echo "Event: $HOOK_EVENT"
    echo "Tool: $TOOL_NAME"
    echo "Args: $TOOL_ARGS"
    echo ""
} >> "$LOG"

# Your hook logic here
exit 0
```

**2. Test Hooks in Isolation**
```bash
# Run hook manually with test variables
export TOOL_NAME="Write"
export TOOL_ARGS='{"file_path": "test.py"}'
.claude/hooks/my-hook.sh
echo "Exit code: $?"
```

**3. Use set -x for Tracing**
```bash
#!/bin/bash
set -x  # Print each command before executing

# Your hook logic
# Each line will be printed to stderr

exit 0
```

**4. Validate JSON Parsing**
```bash
# Test JSON extraction
TEST_ARGS='{"file_path": "/tmp/test.py", "content": "print('hi')"}'
FILE=$(echo "$TEST_ARGS" | grep -o '"file_path":"[^"]*"' | cut -d'"' -f4)
echo "Extracted file: $FILE"
```

### Testing Checklist

Before deploying a hook:

- [ ] Script is executable (`chmod +x`)
- [ ] Path in settings.json is correct
- [ ] Tested with sample environment variables
- [ ] Exit codes are explicit (0 or 1)
- [ ] Error messages are clear and helpful
- [ ] Variables are checked before use
- [ ] Tested both allow and block scenarios

---

## 11. Best Practices for Hooks

### 1. Keep Hooks Fast (<2 seconds)

**Why**: Hooks run on every event. Slow hooks frustrate users.

**Bad** (slow):
```bash
# Run full test suite on every file write
npm test  # Takes 30 seconds
```

**Good** (fast):
```bash
# Only run tests on git commit
if [[ "$TOOL_ARGS" =~ "git commit" ]]; then
    npm test
fi
```

### 2. Provide Clear Error Messages

**Bad**:
```bash
echo "Error"
exit 1
```

**Good**:
```bash
echo "❌ Cannot commit: Tests failed"
echo ""
echo "Failed tests:"
echo "  - test_authentication.py::test_login"
echo ""
echo "Fix tests and try again."
exit 1
```

### 3. Make Hooks Optional

Allow users to bypass hooks when needed:

```bash
# Check for bypass flag
if [ "$CLAUDE_SKIP_HOOKS" = "1" ]; then
    echo "⚠️  Hooks disabled by CLAUDE_SKIP_HOOKS"
    exit 0
fi

# Normal hook logic
```

**Usage**:
```bash
CLAUDE_SKIP_HOOKS=1 claude-code
```

### 4. Log Hook Activity

Track what hooks do for debugging:

```bash
LOG_FILE=".claude/hooks/activity.log"

{
    echo "[$(date '+%Y-%m-%d %H:%M:%S')] Hook: $0"
    echo "  Event: $HOOK_EVENT"
    echo "  Action: [describe what happened]"
    echo ""
} >> "$LOG_FILE"
```

### 5. Don't Require User Input

**Bad**:
```bash
read -p "Confirm deletion (y/n): " confirm
```

Hooks run non-interactively. User input will hang.

**Good**:
```bash
# Just block or allow, no prompts
if [[ dangerous_condition ]]; then
    exit 1  # Block automatically
fi
```

### 6. Handle Missing Dependencies Gracefully

```bash
# Check if command exists
if ! command -v flake8 &>/dev/null; then
    echo "⚠️  flake8 not installed, skipping lint check"
    exit 0  # Don't block if tool missing
fi
```

### 7. Use Consistent Exit Codes

```bash
# At the end of every hook
exit 0  # Allow
# or
exit 1  # Block

# Never exit without code
# Never use other exit codes (2, 3, etc.)
```

### 8. Document Your Hooks

```bash
#!/bin/bash
#
# Hook: Pre-commit linting
# Purpose: Ensure code quality before commits
# Event: tool_call
# Blocks: git commit if linting fails
# Dependencies: flake8, eslint
#

# Hook logic...
```

---

## 12. Real-World Hook Examples

### Example 1: Notebook Validation

Ensure Jupyter notebooks pass quality checks before commit:

```bash
#!/bin/bash

if [[ "$TOOL_ARGS" =~ "git commit" ]]; then
    # Find modified notebooks
    NOTEBOOKS=$(git diff --cached --name-only | grep '.ipynb$')
    
    if [ -n "$NOTEBOOKS" ]; then
        echo "🔍 Validating notebooks..."
        
        for nb in $NOTEBOOKS; do
            # Check execution
            if ! jupyter nbconvert --execute --to notebook "$nb" &>/dev/null; then
                echo "❌ Notebook failed to execute: $nb"
                exit 1
            fi
            
            # Check for outputs (should be stripped)
            if grep -q '"outputs": \[' "$nb"; then
                echo "⚠️  Notebook has outputs: $nb"
                echo "   Run: nbstripout $nb"
                exit 1
            fi
        done
        
        echo "✅ All notebooks validated"
    fi
fi

exit 0
```

### Example 2: Dependency Checking

Ensure required dependencies are installed:

```bash
#!/bin/bash

# Check dependencies before running Python scripts
if [[ "$TOOL_NAME" = "Bash" && "$TOOL_ARGS" =~ "python " ]]; then
    
    # Check if requirements.txt exists
    if [ -f "requirements.txt" ]; then
        # Check if all packages are installed
        while read package; do
            pkg_name=$(echo "$package" | cut -d'=' -f1)
            
            if ! pip show "$pkg_name" &>/dev/null; then
                echo "❌ Missing dependency: $pkg_name"
                echo "   Run: pip install -r requirements.txt"
                exit 1
            fi
        done < requirements.txt
    fi
fi

exit 0
```

### Example 3: Security Scanning

Scan for secrets before committing:

```bash
#!/bin/bash

if [[ "$TOOL_ARGS" =~ "git commit" ]]; then
    echo "🔒 Scanning for secrets..."
    
    # Get staged files
    FILES=$(git diff --cached --name-only)
    
    # Patterns to detect
    PATTERNS=(
        "API_KEY"
        "SECRET_KEY"
        "password\s*=\s*['\"][^'\"]+['\"]"
        "[a-zA-Z0-9]{32,}"  # Long hex strings (tokens)
    )
    
    for file in $FILES; do
        for pattern in "${PATTERNS[@]}"; do
            if grep -q -i "$pattern" "$file"; then
                echo "❌ Potential secret found in $file"
                echo "   Pattern: $pattern"
                echo ""
                echo "   Review the file and remove secrets."
                exit 1
            fi
        done
    done
    
    echo "✅ No secrets detected"
fi

exit 0
```

### Example 4: Code Formatting

Auto-format on save:

```bash
#!/bin/bash

if [[ "$TOOL_NAME" = "Write" || "$TOOL_NAME" = "Edit" ]]; then
    FILE=$(echo "$TOOL_ARGS" | jq -r '.file_path')
    
    # Format Python files
    if [[ "$FILE" =~ \.py$ ]]; then
        black --line-length 100 "$FILE" &>/dev/null
        isort "$FILE" &>/dev/null
        echo "✨ Formatted: $FILE"
    fi
    
    # Format JavaScript files
    if [[ "$FILE" =~ \.js$ ]]; then
        prettier --write "$FILE" &>/dev/null
        echo "✨ Formatted: $FILE"
    fi
fi

exit 0
```

### Example 5: Test Coverage Enforcement

Require minimum test coverage:

```bash
#!/bin/bash

if [[ "$TOOL_ARGS" =~ "git commit" ]]; then
    echo "📊 Checking test coverage..."
    
    # Run tests with coverage
    COVERAGE=$(pytest --cov --cov-report=term-missing | grep "TOTAL" | awk '{print $4}' | sed 's/%//')
    
    MIN_COVERAGE=80
    
    if [ "$COVERAGE" -lt "$MIN_COVERAGE" ]; then
        echo "❌ Test coverage too low: ${COVERAGE}%"
        echo "   Minimum required: ${MIN_COVERAGE}%"
        echo ""
        echo "   Add tests to increase coverage."
        exit 1
    fi
    
    echo "✅ Coverage: ${COVERAGE}% (meets ${MIN_COVERAGE}% threshold)"
fi

exit 0
```

---

## 13. Practice Exercises

### Exercise 9: Complete Pre-Commit Workflow ⭐⭐⭐

**Goal**: Build a comprehensive quality gate system.

**Task**: Create `.claude/hooks/comprehensive-check.sh` that:

1. **Validates code formatting**
   - Python: black, isort
   - JavaScript: prettier
   
2. **Runs linters**
   - Python: flake8
   - JavaScript: eslint
   
3. **Executes tests**
   - Run full test suite
   - Check for failures
   
4. **Scans for issues**
   - TODO/FIXME comments
   - Debug statements (console.log, print)
   - Secrets (API keys)

**Requirements**:
- Clear progress indicators for each step
- Helpful error messages
- Exit early if any check fails
- Only run on `git commit` commands

### Exercise 10: Security Validation Hook ⭐⭐⭐

**Goal**: Prevent security vulnerabilities from being committed.

**Task**: Create a hook that scans for:
- Hardcoded passwords
- API keys and tokens
- Private keys (RSA, SSH)
- AWS credentials
- Database connection strings with passwords

**Bonus**: Use regex patterns to detect common secret formats.

### Exercise 11: Documentation Sync Hook ⭐⭐

**Goal**: Keep documentation in sync with code.

**Task**: Create a hook that:
1. Detects when Python files are modified
2. Checks if corresponding documentation exists
3. If docs are missing or outdated, remind the developer
4. Optionally: Auto-generate docstrings

### Exercise 12: Error Prevention Hooks ⭐⭐⭐

**Goal**: Prevent common mistakes.

**Task**: Create hooks that block:

1. **Committing to main/master branch**
   - Check current branch
   - Block if on main/master
   - Suggest creating feature branch

2. **Large file commits**
   - Check file sizes
   - Block files >10MB
   - Suggest using Git LFS

3. **Incomplete features**
   - Check for TODO/FIXME in modified files
   - Warn about incomplete work
   - Require confirmation

---

## 14. Hook Testing Framework

Let's create a framework to test our hooks:

In [ ]:
import subprocess
import os
from pathlib import Path

class HookTester:
    """Test framework for Claude Code hooks."""
    
    def __init__(self, hook_path):
        self.hook_path = Path(hook_path)
        if not self.hook_path.exists():
            raise FileNotFoundError(f"Hook not found: {hook_path}")
    
    def test(self, event, tool_name=None, tool_args=None, prompt=None):
        """Test hook with given environment."""
        
        env = os.environ.copy()
        env['HOOK_EVENT'] = event
        
        if tool_name:
            env['TOOL_NAME'] = tool_name
        if tool_args:
            env['TOOL_ARGS'] = tool_args
        if prompt:
            env['PROMPT'] = prompt
        
        result = subprocess.run(
            [str(self.hook_path)],
            env=env,
            capture_output=True,
            text=True
        )
        
        return {
            'exit_code': result.returncode,
            'stdout': result.stdout,
            'stderr': result.stderr,
            'blocked': result.returncode != 0
        }
    
    def run_test_suite(self, tests):
        """Run multiple tests and report results."""
        
        print(f"🧪 Testing Hook: {self.hook_path.name}\n")
        print("=" * 70)
        
        passed = 0
        failed = 0
        
        for test in tests:
            name = test['name']
            expected_block = test.get('should_block', False)
            
            result = self.test(
                test.get('event', 'tool_call'),
                test.get('tool_name'),
                test.get('tool_args'),
                test.get('prompt')
            )
            
            # Check if result matches expectation
            success = result['blocked'] == expected_block
            
            if success:
                print(f"\n✅ PASS: {name}")
                passed += 1
            else:
                print(f"\n❌ FAIL: {name}")
                print(f"   Expected: {'BLOCK' if expected_block else 'ALLOW'}")
                print(f"   Got: {'BLOCK' if result['blocked'] else 'ALLOW'}")
                failed += 1
            
            if result['stdout']:
                print(f"   Output: {result['stdout'][:100]}...")
        
        print("\n" + "=" * 70)
        print(f"\n📊 Results: {passed} passed, {failed} failed")
        
        return passed, failed

# Example usage
if Path('.claude/hooks/protect-files.sh').exists():
    tester = HookTester('.claude/hooks/protect-files.sh')
    
    test_suite = [
        {
            'name': 'Block write to production.json',
            'tool_name': 'Write',
            'tool_args': '{"file_path": "production.json"}',
            'should_block': True
        },
        {
            'name': 'Allow write to development.json',
            'tool_name': 'Write',
            'tool_args': '{"file_path": "development.json"}',
            'should_block': False
        },
        {
            'name': 'Block edit to secrets.env',
            'tool_name': 'Edit',
            'tool_args': '{"file_path": "secrets.env"}',
            'should_block': True
        },
    ]
    
    tester.run_test_suite(test_suite)
else:
    print("⚠️  Create .claude/hooks/protect-files.sh first!")

Use this framework to test any hook you create!

---

## 15. Summary

### What You've Learned

In this module, you mastered:

✅ **Hook Fundamentals**
- Event-driven automation in Claude Code
- Three hook types: tool_call, user_prompt_submit, assistant_message
- Exit codes: 0 (allow) vs 1 (block)

✅ **Hook Types**
- **Logging hooks**: Record activity (always exit 0)
- **Validation hooks**: Block dangerous operations (exit 1)
- **Quality gates**: Enforce standards before commits
- **Auto-formatting**: Automatically improve code

✅ **Environment Variables**
- `$HOOK_EVENT`: Event type
- `$TOOL_NAME`: Tool being used
- `$TOOL_ARGS`: Tool arguments (JSON)
- Parsing JSON with grep or jq

✅ **Real-World Patterns**
- File protection (block writes to critical files)
- Pre-commit workflows (linting, testing)
- Security scanning (secrets detection)
- Dependency checking
- Test coverage enforcement

✅ **Best Practices**
- Keep hooks fast (<2 seconds)
- Clear, actionable error messages
- Graceful handling of missing dependencies
- Logging for debugging
- No user input required

✅ **Debugging Skills**
- Test hooks in isolation
- Use logging for troubleshooting
- Validate JSON parsing
- Testing framework for hooks

### Key Takeaways

1. **Hooks enable automation** - Run scripts automatically on events
2. **Exit codes control behavior** - 0 allows, 1 blocks
3. **Environment variables provide context** - Access tool names, arguments
4. **Quality gates prevent mistakes** - Catch issues before they're committed
5. **Best practices matter** - Fast, clear, optional hooks work best

### Hooks vs Commands vs Skills

| Feature | Hooks | Commands | Skills |
|---------|-------|----------|--------|
| **Trigger** | Automatic (events) | Manual (`/cmd`) | Automatic (keywords) |
| **Purpose** | Validate, automate | Execute prompts | Provide expertise |
| **Can block?** | ✅ Yes | ❌ No | ❌ No |
| **Best for** | Quality gates, safety | Repeatable tasks | Domain knowledge |

---

## 16. What's Next?

In **Module 06: MCP Servers and Integrations**, you'll learn:

- 🔌 **Model Context Protocol (MCP)** fundamentals
- 🌐 **External service integration** (GitHub, databases, APIs)
- ⚙️ **Installing and configuring** MCP servers
- 🔒 **Security best practices** for credentials
- 🛠️ **Common MCP servers** and their use cases

MCP extends Claude Code's capabilities by connecting to external services!

---

## 17. Additional Resources

### Official Documentation
- [Claude Code Hooks Guide](https://docs.anthropic.com/claude-code/hooks)
- [Hook Configuration Reference](https://docs.anthropic.com/claude-code/configuration#hooks)
- [Security Best Practices](https://docs.anthropic.com/claude-code/security)

### Community Resources
- [Awesome Claude Code Hooks](https://github.com/topics/claude-code-hooks) - Community hook collections
- [Hook Examples Repository](https://github.com/anthropics/claude-code-examples)

### Tools and Utilities
- **black**: Python code formatter
- **flake8**: Python linter
- **prettier**: JavaScript formatter
- **eslint**: JavaScript linter
- **jq**: JSON processor for bash

### Related Modules
- **Module 03**: [Working with Skills](03_working_with_skills.ipynb)
- **Module 04**: [Custom Slash Commands](04_custom_slash_commands.ipynb)
- **Module 06**: [MCP Servers](06_mcp_servers_integrations.ipynb) (coming next)
- **Module 08**: [Advanced Workflows](08_advanced_workflows.ipynb) (combining everything)

---

## 🎉 Congratulations!

You've completed Module 05 and can now:
- Create powerful automation hooks
- Build validation and quality gates
- Implement pre-commit workflows
- Debug hook issues effectively
- Follow hook best practices

**Next Step**: Move to Module 06 to learn about MCP servers and integrations!

---

*Claude Code Mastery Series - Module 05 of 10*